In [1]:
##########################################
# NACA 0012 DNS post-processing shirnk data set size so can be download to local and do-postprocessing.         #
# Zhenyang Yuan                          #
# 25/5/2021                              #
##########################################
import numpy as np
from math import pi
import math
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
import scipy.signal
from numpy import linalg as LA
#from matplotlib.mlab import griddata
params = {'legend.fontsize': 15,
          'legend.loc':'best',
          'figure.figsize': (14,5),
          'lines.markerfacecolor':'none',
         'axes.labelsize': 17,
         'axes.titlesize': 17,
         'xtick.labelsize':15,
         'ytick.labelsize':15,
         'grid.alpha':0.6}
pylab.rcParams.update(params)
#%matplotlib notebook
%matplotlib inline


from sys import argv

import pandas as pd

In [2]:
# load data set
def data_load_DNS(time,t,start,end,x_,y_,headline,varibles,numNodes,Lines):
    r = 5
    #x_ = np.asarray([])
    #y_ = np.asarray([])
    p_ = np.asarray([])

    x = np.asarray([])
    y = np.asarray([])
    p = np.asarray([])
    
    
        
    

    k = 0
    #for i in open("../send/"+str(time)+".dat").readlines():
    for i in open("/Users/yuanzhenyang/software/yuan_compute/aerofoil/postprocessing/walldatFiles/bnd_wall_"+str(time)+".dat").readlines():
        if k > headline - 1:                                #get away with headers
            datContent = i.strip().split()                  #magic

            datContent = np.asarray(datContent)             #convert list to array
            datContent = datContent.astype(np.float64)      #convert str element to float

            if t == 0:
                if k < Lines*1 + headline :
                        x_ = np.concatenate((x_,datContent),axis=None)
                elif k < Lines*2 + headline :
                        y_ = np.concatenate((y_,datContent),axis=None)
                elif k < Lines*7 + headline and k > Lines*6 + headline - 1:
                        u_ = np.concatenate((u_,datContent),axis=None)
                elif k < Lines*8 + headline and k > Lines*7 + headline - 1:
                        v_ = np.concatenate((v_,datContent),axis=None)
                elif k < Lines*9 + headline and k > Lines*8 + headline - 1:
                        p_ = np.concatenate((p_,datContent),axis=None)
            else:
                if k < Lines*7 + headline and k > Lines*6 + headline - 1:
                        p_ = np.concatenate((p_,datContent),axis=None)
                elif k < Lines*8 + headline and k > Lines*7 + headline - 1:
                        p_ = np.concatenate((p_,datContent),axis=None)
                elif k < Lines*9 + headline and k > Lines*8 + headline - 1:
                        p_ = np.concatenate((p_,datContent),axis=None)
    


        k+=1
        if k > Lines*9 + headline:
            break

    for i in range(len(x_)):
        if abs(x_[i]) < r and abs(y_[i]) < r:
            if t == 0:
                x = np.concatenate((x,x_[i]),axis=None)
                y = np.concatenate((y,y_[i]),axis=None)
                p = np.concatenate((p,p_[i]),axis=None)
            else:
                p = np.concatenate((p,p_[i]),axis=None)

    print('Completed: '+str(time-start+1)+'/'+str(end-start+1)+'\n')

    if t == 0:
        return x,y,p,x_,y_
    elif time == end:
        print('Completed collecting data from Nektar++ DNS solution!')
        return p
    else:
        return p


In [3]:
start = 27    #27
end = 107      #67
time = np.arange(start,end+1,1)

headline=3
varibles=3   # x-1, y-2, rho-3, rhou-4, rhov-5, E-6, u-7, v-8, p-9, T, s, a, Mach, Sensor
numNodes=888
#Lines=int(numNodes/5) #from tecplot
Lines=int(numNodes/1000)+1 #fieldconvert


x_ = np.asarray([])   #pruely for loading speed reason
y_ = np.asarray([])

xwall,ywall,p1wall,x_,y_ = data_load_DNS(time[0],0,start,end,x_,y_,headline,varibles,numNodes,Lines)

pwall = np.zeros([len(time), len(p1wall)])
pwall[0] = p1wall
for i in range(len(time)):
    if i > 0:
        pwall[i][:]=data_load_DNS(time[i],1,start,end,x_,y_,headline,varibles,numNodes,Lines)


Completed: 1/81

Completed: 2/81

Completed: 3/81

Completed: 4/81

Completed: 5/81

Completed: 6/81

Completed: 7/81

Completed: 8/81

Completed: 9/81

Completed: 10/81

Completed: 11/81

Completed: 12/81

Completed: 13/81

Completed: 14/81

Completed: 15/81

Completed: 16/81

Completed: 17/81

Completed: 18/81

Completed: 19/81

Completed: 20/81

Completed: 21/81

Completed: 22/81

Completed: 23/81

Completed: 24/81

Completed: 25/81

Completed: 26/81

Completed: 27/81

Completed: 28/81

Completed: 29/81

Completed: 30/81

Completed: 31/81

Completed: 32/81

Completed: 33/81

Completed: 34/81

Completed: 35/81

Completed: 36/81

Completed: 37/81

Completed: 38/81

Completed: 39/81

Completed: 40/81

Completed: 41/81

Completed: 42/81

Completed: 43/81

Completed: 44/81

Completed: 45/81

Completed: 46/81

Completed: 47/81

Completed: 48/81

Completed: 49/81

Completed: 50/81

Completed: 51/81

Completed: 52/81

Completed: 53/81

Completed: 54/81

Completed: 55/81

Completed: 56/81

C

In [4]:
def write2dat_coord(x,y):
    f.open('coordinate.dat','a')
    f.write('Coordinate varibles\n')
    f.write('x\n')
    f.write('y\n')
    
    
    f.write(str(x)+'\n')
    f.write(str(y)+'\n')
    
    f.close()

In [6]:
def write2dat_fluid(time,u,v,p):
    f.open(str(time)+'fluid.dat','a')
    f.write('fluid data, physical time:'+str((time-30)*0.02)+'\n')
    f.write('Fluid varibles\n')
    f.write('p\n')    
    f.write('u\n')        
    f.write('v\n')    
    
    f.write(str(p)+'\n')
    f.write(str(u)+'\n')
    f.write(str(v)+'\n')    
    
    f.close()    